In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/17 04:11:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")
#df.printSchema()

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

In [3]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.ml import Pipeline 
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import StringIndexer 
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexers = [
StringIndexer(inputCol="Flow ID", outputCol = "flowid_index"),  
StringIndexer(inputCol="Dst IP", outputCol = "dstip_index"),
StringIndexer(inputCol="FIN Flag Cnt", outputCol = "fin_index"),
StringIndexer(inputCol="SYN Flag Cnt", outputCol = "syn_index"),
StringIndexer(inputCol="URG Flag Cnt", outputCol = "urg_index"),
StringIndexer(inputCol="PSH Flag Cnt", outputCol = "cwe_index"),
StringIndexer(inputCol="ECE Flag Cnt", outputCol = "ece_index"),
StringIndexer(inputCol="Label", outputCol = "label")]

In [5]:
pipeline = Pipeline(stages=indexers)
df = df.select("Flow ID",   
               "Dst IP", 
               "FIN Flag Cnt", 
               "SYN Flag Cnt", 
               "URG Flag Cnt",
               "PSH Flag Cnt",
               "ECE Flag Cnt",
               "Label"
              )
df = df.sample(fraction=0.05)
df1 = df.toPandas()
df.unpersist()
indexed_rows_df = Pipeline.fit(df1).transform(df1)
indexed_rows_df.show(5, False)

TypeError: fit() missing 1 required positional argument: 'dataset'

In [ ]:
vectorAssembler = VectorAssembler(inputCols = ["flowid_index", "dstip_index","fin_index", "syn_index", "urg_index", "cwe_index","ece_index"],outputCol = "features")
vindexed_rows_df = VectorAssembler.transform(indexed_rows_df)
vindexed_rows_df.show(5, False)

In [ ]:
splits = vindexed_rows_df.randomSplit([0.6,0.4], 42) 
# optional value 42 is seed for sampling 
train_df = splits[0] 
test_df = splits[1]

In [ ]:
nb = NaiveBayes(modelType="multinomial")
nbmodel = nb.fit(train_df)

In [ ]:
predictions_df = nbmodel.transform(test_df)
predictions_df.show(5, True)

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="pr ediction", metricName="accuracy") 
nbaccuracy = evaluator.evaluate(predictions_df) 
print("Test accuracy = " + str(nbaccuracy))